In [ ]:
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt
import cv2
import numpy as np
import math

#Read all images from folder
pospath = "/home/collin/Desktop/Positives"
positives = [f for f in listdir(pospath) if isfile(join(pospath, f))]

negpath = "/home/collin/Desktop/Negatives"
negatives = [f for f in listdir(negpath) if isfile(join(negpath, f))]

#create array to hold all data
numImages = len(positives) + len(negatives)
data = np.zeros((numImages, 1025))

#choose 80/20 ratio for splitting data because of Pareto principle
split = math.floor(numImages * 0.8)

#load Positives
for i in range(len(positives)):
    path = "/home/collin/Desktop/Positives/" + positives[i]
    particle = cv2.imread(path)
    particle = cv2.cvtColor(particle, cv2.COLOR_BGR2GRAY)
    
    particle = particle.flatten()
    particle = np.insert(particle, 1024, 1)
    data[i] = particle
    
#load Negatives
for i in range(len(negatives)):
    path = "/home/collin/Desktop/Negatives/" + negatives[i]
    particle = cv2.imread(path)
    particle = cv2.cvtColor(particle, cv2.COLOR_BGR2GRAY)
    
    particle = particle.flatten()
    particle = np.insert(particle, 1024, 1)
    data[len(positives) + i] = particle

#shuffle examples
np.random.shuffle(data)

#split data into training and testing sets
X_train = data[0:split, 0:1024]
y_train = data[0:split, 1024]
X_test = data[split:,0:1024]
y_test = data[split:, 1024]
print X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [7]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape

(60000, 28, 28)

In [5]:
if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 55s - loss: 0.3822 - acc: 0.8822 - val_loss: 0.0857 - val_acc: 0.9737
Epoch 2/12
60000/60000 [==============================] - 55s - loss: 0.1340 - acc: 0.9601 - val_loss: 0.0618 - val_acc: 0.9799
Epoch 3/12
60000/60000 [==============================] - 55s - loss: 0.1050 - acc: 0.9685 - val_loss: 0.0543 - val_acc: 0.9823
Epoch 4/12
60000/60000 [==============================] - 55s - loss: 0.0874 - acc: 0.9732 - val_loss: 0.0450 - val_acc: 0.9847
Epoch 5/12
60000/60000 [==============================] - 55s - loss: 0.0788 - acc: 0.9768 - val_loss: 0.0415 - val_acc: 0.9862
Epoch 6/12
60000/60000 [==============================] - 56s - loss: 0.0706 - acc: 0.9794 - val_loss: 0.0390 - val_acc: 0.9869
Epoch 7/12
60000/60000 [==============================] - 57s - loss: 0.0632 - acc: 0.9812 - val_loss: 0.0374